[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/trainNNmodel.ipynb)

In [1]:
!pip install tqdm
from tqdm import tqdm

In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from keras import models
from keras import layers

from keras.models import load_model


Using TensorFlow backend.


In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/
%ls

/content/gdrive/My Drive/projects
model_LGB.sav  model_NN.h5  model_XGB.sav  user_train_wip.csv


In [0]:
s_data = 'user_train_wip.csv'
s_col_tgt = 'totals.transactionRevenue_sum'
l_col_drop = ['date_min', 'date_max',s_col_tgt]
filename = 'model_NN.h5'

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 10.4 s, sys: 1.99 s, total: 12.4 s
Wall time: 14.4 s


In [7]:
%%time
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_x_n = pd.DataFrame(sc.fit_transform(df_x.fillna(0)),columns=df_x.columns)
df_x_n

CPU times: user 2.26 s, sys: 1.13 s, total: 3.39 s
Wall time: 3.45 s


In [0]:
def build_model(N_col):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(N_col,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [8]:
%%time
epochs = 20
batch_size = 350000

model = build_model(df_x.shape[1])

history = model.fit(df_x.fillna(0),#df_x_n
                    df_y,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=1)

Epoch 1/20
714167/714167 [==============================] - 4s 6us/step - loss: 6254739037930.0752 - mean_absolute_error: 228334.2973
Epoch 2/20
714167/714167 [==============================] - 3s 4us/step - loss: 5913788517.3457 - mean_absolute_error: 8576.5387
Epoch 3/20
714167/714167 [==============================] - 3s 4us/step - loss: 59966519.4686 - mean_absolute_error: 778.5649
Epoch 4/20
714167/714167 [==============================] - 3s 4us/step - loss: 54555258.0783 - mean_absolute_error: 697.7294
Epoch 5/20
714167/714167 [==============================] - 3s 4us/step - loss: 1580600409.0068 - mean_absolute_error: 3762.4957
Epoch 6/20
714167/714167 [==============================] - 3s 4us/step - loss: 40377330942.4475 - mean_absolute_error: 25061.3812
Epoch 7/20
714167/714167 [==============================] - 3s 4us/step - loss: 147433094281.4062 - mean_absolute_error: 47177.8314
Epoch 8/20
714167/714167 [==============================] - 3s 4us/step - loss: 632134555664.

In [15]:
history.history

{'loss': [8.564659477529949,
  7.46198294669111,
  6.934467788110956,
  6.564304266775209,
  6.270706642836594,
  6.058307173662174,
  5.906127979676258,
  5.7581761899735096,
  5.755019950471297,
  5.652122472863003],
 'mean_absolute_error': [0.6080875236581412,
  0.5780460474334104,
  0.6057166252831829,
  0.6067252476955787,
  0.5999682404565487,
  0.5918505922535273,
  0.5818488258317185,
  0.5593260598749341,
  0.5811313638354567,
  0.5535816328732629]}

In [9]:
%%time
model.save(filename)

CPU times: user 49.9 ms, sys: 8.31 ms, total: 58.2 ms
Wall time: 561 ms


In [0]:
files.download(filename)